# 중규모 시뮬레이션

우리는 이전까지 페르소나를 생성하고 API작동 및 프롬프트 성능 체크를 위한 소규모 LLM을 구성하고 테스트를 진행하였다. 결과는 좋았고 이번에는 통계적으로 유의한 패턴 확보를 위해 중규모 시뮬레이션을 할 예정이다.

__📋 진행 계획__
```
목표: 카테고리별 25개 × 4 = 100개 페르소나 시뮬레이션
시간: 2-3시간
비용: $20-40
결과: 통계적으로 유의한 패턴 확보
```

In [1]:
# 프로젝트 초기화
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import openai
from openai import OpenAI
from google.colab import userdata
from datetime import datetime
from collections import Counter, defaultdict
from dataclasses import dataclass, asdict
import re
import warnings
warnings.filterwarnings('ignore')

## 데이터 로드

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Dongwon/data/raw/product_info.csv")

## 페르소나 속성 클래스 재정의

In [4]:
# 페르소나 속성 클래스 재정의
@dataclass
class PersonaAttributes:
    age_group: str
    gender: str
    income_level: str
    region: str
    family_type: str
    occupation: str
    shopping_style: str
    health_concern: str
    brand_preference: str
    price_sensitivity: str
    product_category: str = ""
    confidence_level: str = ""
    persona_id: str = ""

    def to_dict(self):
        return asdict(self)

# 이전 분석 결과 요약
ANALYSIS_SUMMARY = {
    "portfolio_strategy": {
        "조미소스": {"count": 6, "strategy": "무투자 물량공세", "confidence": "고확실성"},
        "참치": {"count": 4, "strategy": "고투자 프리미엄", "confidence": "중확실성"},
        "우유류": {"count": 3, "strategy": "중간투자 트렌드", "confidence": "중확실성"},
        "축산캔": {"count": 2, "strategy": "실험적 신개념", "confidence": "저확실성"}
    }
}

# 카테고리별 페르소나 전략
PERSONA_STRATEGIES = {
    "조미소스": {
        "confidence_level": "high_confidence",
        "conservatism_level": "moderate",
        "target_description": "기존 동원 고객 + 요리 애호가 주부층"
    },
    "참치": {
        "confidence_level": "medium_confidence",
        "conservatism_level": "high",
        "target_description": "안유진 팬덤 + 건강 관심 젊은층"
    },
    "우유류": {
        "confidence_level": "medium_confidence",
        "conservatism_level": "very_high",
        "target_description": "건강 트렌드 민감층 + 락토프리 필요층"
    },
    "축산캔": {
        "confidence_level": "low_confidence",
        "conservatism_level": "extreme",
        "target_description": "얼리어답터 + 간편식 선호층"
    }
}

print("✅ 전략 설정 로드 완료")

✅ 전략 설정 로드 완료


In [5]:
# API 키 설정
api_key = userdata.get('openai_api_key')
client = OpenAI(api_key=api_key)

## 중규모 시뮬레이션용 페르소나 생성

In [6]:
print("\n👥 중규모 시뮬레이션용 페르소나 생성")
print("="*60)

# 1. 속성 풀 재정의 (이전과 동일하지만 간소화)
ATTRIBUTE_POOLS = {
    "age_group": {
        "20-29세": {"weight": 25, "purchase_propensity": 0.7},
        "30-39세": {"weight": 30, "purchase_propensity": 0.8},
        "40-49세": {"weight": 25, "purchase_propensity": 0.6},
        "50-59세": {"weight": 15, "purchase_propensity": 0.5},
        "60세이상": {"weight": 5, "purchase_propensity": 0.3}
    },
    "gender": {
        "여성": {"weight": 55, "purchase_propensity": 0.7},
        "남성": {"weight": 45, "purchase_propensity": 0.6}
    },
    "income_level": {
        "200만원미만": {"weight": 15, "purchase_propensity": 0.3},
        "200-400만원": {"weight": 35, "purchase_propensity": 0.5},
        "400-600만원": {"weight": 30, "purchase_propensity": 0.7},
        "600만원이상": {"weight": 20, "purchase_propensity": 0.8}
    },
    "region": {
        "서울": {"weight": 20, "purchase_propensity": 0.8},
        "경기": {"weight": 25, "purchase_propensity": 0.7},
        "부산": {"weight": 8, "purchase_propensity": 0.6},
        "기타광역시": {"weight": 20, "purchase_propensity": 0.6},
        "기타지역": {"weight": 27, "purchase_propensity": 0.5}
    },
    "family_type": {
        "1인가구": {"weight": 30, "purchase_propensity": 0.6},
        "신혼부부": {"weight": 15, "purchase_propensity": 0.8},
        "자녀있는가정": {"weight": 40, "purchase_propensity": 0.7},
        "노부부": {"weight": 15, "purchase_propensity": 0.4}
    },
    "occupation": {
        "사무직": {"weight": 35, "purchase_propensity": 0.7},
        "전문직": {"weight": 15, "purchase_propensity": 0.8},
        "주부": {"weight": 20, "purchase_propensity": 0.6},
        "학생": {"weight": 10, "purchase_propensity": 0.5},
        "자영업": {"weight": 12, "purchase_propensity": 0.6},
        "기타": {"weight": 8, "purchase_propensity": 0.5}
    },
    "shopping_style": {
        "신중형": {"weight": 40, "purchase_propensity": 0.5},
        "충동형": {"weight": 20, "purchase_propensity": 0.8},
        "가성비형": {"weight": 25, "purchase_propensity": 0.6},
        "브랜드형": {"weight": 15, "purchase_propensity": 0.7}
    },
    "health_concern": {
        "높음": {"weight": 30, "purchase_propensity": 0.7},
        "보통": {"weight": 50, "purchase_propensity": 0.6},
        "낮음": {"weight": 20, "purchase_propensity": 0.5}
    },
    "brand_preference": {
        "강함": {"weight": 25, "purchase_propensity": 0.7},
        "보통": {"weight": 50, "purchase_propensity": 0.6},
        "약함": {"weight": 25, "purchase_propensity": 0.6}
    },
    "price_sensitivity": {
        "높음": {"weight": 35, "purchase_propensity": 0.4},
        "보통": {"weight": 45, "purchase_propensity": 0.6},
        "낮음": {"weight": 20, "purchase_propensity": 0.8}
    }
}


👥 중규모 시뮬레이션용 페르소나 생성


In [7]:
# 중규모 페르소나 생성기
import random

def generate_medium_scale_personas(category, num_personas=25):
    """중규모 시뮬레이션용 페르소나 생성"""
    personas = []

    # 카테고리별 속성 조정 (간소화된 버전)
    adjusted_pools = ATTRIBUTE_POOLS.copy()

    if category == "조미소스":
        adjusted_pools["age_group"]["40-49세"]["weight"] *= 1.5
        adjusted_pools["gender"]["여성"]["weight"] *= 1.3
        adjusted_pools["occupation"]["주부"]["weight"] *= 1.8
    elif category == "참치":
        adjusted_pools["age_group"]["20-29세"]["weight"] *= 1.8
        adjusted_pools["gender"]["여성"]["weight"] *= 1.5
        adjusted_pools["shopping_style"]["브랜드형"]["weight"] *= 1.5
    elif category == "우유류":
        adjusted_pools["age_group"]["30-39세"]["weight"] *= 1.5
        adjusted_pools["health_concern"]["높음"]["weight"] *= 1.8
    elif category == "축산캔":
        adjusted_pools["family_type"]["1인가구"]["weight"] *= 1.5
        adjusted_pools["shopping_style"]["충동형"]["weight"] *= 1.3

    for i in range(num_personas):
        persona_attrs = {}

        for attr_name, attr_pool in adjusted_pools.items():
            choices = list(attr_pool.keys())
            weights = [attr_pool[choice]["weight"] for choice in choices]
            selected = random.choices(choices, weights=weights)[0]
            persona_attrs[attr_name] = selected

        persona = PersonaAttributes(
            **persona_attrs,
            product_category=category,
            confidence_level=PERSONA_STRATEGIES[category]["confidence_level"],
            persona_id=f"medium_{category}_{i+1:03d}"
        )

        personas.append(persona)

    return personas

In [8]:
# 카테고리별 페르소나 생성
medium_personas = {}
total_personas = 0

for category in ["조미소스", "참치", "우유류", "축산캔"]:
    print(f"\n🎯 {category} 페르소나 25개 생성 중...")
    personas = generate_medium_scale_personas(category, 25)
    medium_personas[category] = personas
    total_personas += len(personas)
    print(f"✅ {category}: {len(personas)}개 생성 완료")

print(f"\n✅ 총 페르소나 생성 완료: {total_personas}개")


🎯 조미소스 페르소나 25개 생성 중...
✅ 조미소스: 25개 생성 완료

🎯 참치 페르소나 25개 생성 중...
✅ 참치: 25개 생성 완료

🎯 우유류 페르소나 25개 생성 중...
✅ 우유류: 25개 생성 완료

🎯 축산캔 페르소나 25개 생성 중...
✅ 축산캔: 25개 생성 완료

✅ 총 페르소나 생성 완료: 100개


## LLM 시뮬레이션 시스템 구축

In [9]:
# ===== LLM 시뮬레이션 시스템 =====
print("\n🤖 LLM 시뮬레이션 시스템 구축")
print("="*60)

# 1. 프롬프트 템플릿 재구성 (간소화)
class MediumScalePromptGenerator:
    """중규모 시뮬레이션용 프롬프트 생성기"""

    def __init__(self):
        self.system_message = """당신은 한국 소비자 행동 분석 전문가입니다.
신제품 시장 수용 패턴을 현실적이고 보수적으로 예측합니다.

원칙:
- 신제품은 초기 관망세가 있습니다
- 과도한 낙관론을 지양합니다
- 한국 소비자 특성을 반영합니다
- 계절성과 경쟁을 고려합니다"""

    def create_prompt(self, persona, product_name, category):
        """개별 프롬프트 생성"""

        # 보수성 지시사항
        conservatism_map = {
            "moderate": "신중하지만 균형잡힌 관점에서",
            "high": "보수적이고 신중한 관점에서",
            "very_high": "매우 보수적이고 현실적인 관점에서",
            "extreme": "극도로 보수적이고 회의적인 관점에서"
        }

        conservatism = conservatism_map[PERSONA_STRATEGIES[category]["conservatism_level"]]

        user_message = f"""
# 소비자 프로필
연령/성별: {persona.age_group} {persona.gender}
소득/지역: {persona.income_level}, {persona.region}
가족: {persona.family_type}, 직업: {persona.occupation}
성향: {persona.shopping_style}, 건강관심: {persona.health_concern}
브랜드선호: {persona.brand_preference}, 가격민감도: {persona.price_sensitivity}

# 신제품 정보
제품명: {product_name}
카테고리: {category}
출시: 2024년 7월

# 예측 요청
{conservatism} 이 소비자의 12개월 구매 패턴을 예측하세요.

중요 고려사항:
- 신제품 수용의 어려움을 현실적으로 반영
- 월별 변동성과 계절성 고려
- 경제적 부담과 필요성 균형

JSON 응답:
{{
    "monthly_purchase_probability": [월1, 월2, ..., 월12],  // 0-100%
    "monthly_purchase_frequency": [월1, 월2, ..., 월12],   // 0-3회
    "reasoning": "예측 근거",
    "confidence_level": 0.0-1.0
}}
"""

        return {
            "system": self.system_message,
            "user": user_message,
            "metadata": {
                "persona_id": persona.persona_id,
                "product_name": product_name,
                "category": category
            }
        }


🤖 LLM 시뮬레이션 시스템 구축


In [14]:
# 2. 배치 처리 LLM 클라이언트
class MediumScaleLLMClient:
    """중규모 시뮬레이션용 LLM 클라이언트"""

    def __init__(self):
        self.client = client  # 이미 초기화된 OpenAI 클라이언트
        self.results = []
        self.total_cost = 0
        self.total_time = 0
        self.failed_calls = 0

    def call_llm_batch(self, prompts, batch_size=5):
        """배치 단위로 LLM 호출"""
        total_prompts = len(prompts)
        print(f"📞 총 {total_prompts}개 프롬프트를 {batch_size}개씩 배치 처리")

        for i in range(0, total_prompts, batch_size):
            batch = prompts[i:i+batch_size]
            print(f"\n📦 배치 {i//batch_size + 1}: {len(batch)}개 처리 중...")

            for j, prompt_data in enumerate(batch):
                current_idx = i + j + 1
                print(f"  [{current_idx}/{total_prompts}] {prompt_data['metadata']['category']} 처리 중...", end=" ")

                start_time = time.time()

                try:
                    response = self.client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "system", "content": prompt_data["system"]},
                            {"role": "user", "content": prompt_data["user"]}
                        ],
                        temperature=0.7,
                        max_tokens=1500,
                        timeout=60
                    )

                    end_time = time.time()
                    response_time = end_time - start_time

                    content = response.choices[0].message.content
                    cost = (response.usage.prompt_tokens * 0.0015 + response.usage.completion_tokens * 0.002) / 1000

                    # 응답 파싱
                    parsed = self.parse_response(content)

                    if parsed["success"]:
                        result = {
                            "metadata": prompt_data["metadata"],
                            "response": parsed["data"],
                            "api_info": {
                                "response_time": response_time,
                                "cost": cost,
                                "tokens": response.usage.total_tokens
                            },
                            "raw_content": content
                        }
                        self.results.append(result)
                        self.total_cost += cost
                        self.total_time += response_time
                        print(f"✅ 성공 (${cost:.4f})")
                    else:
                        print(f"❌ 파싱실패: {parsed['error']}")
                        self.failed_calls += 1

                except Exception as e:
                    print(f"❌ API 실패: {str(e)[:50]}...")
                    self.failed_calls += 1
                    end_time = time.time()
                    self.total_time += (end_time - start_time)

                # Rate limiting 방지
                time.sleep(0.5)

            # 배치 간 휴식
            if i + batch_size < total_prompts:
                print(f"⏸️ 배치 완료. 5초 휴식...")
                time.sleep(5)

                # 중간 저장
                self.save_intermediate_results(f"batch_{i//batch_size + 1}")

        print(f"\n🎉 전체 배치 처리 완료!")
        return self.results

    def parse_response(self, content):
        """응답 파싱"""
        try:
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                data = json.loads(json_match.group())
                if self.validate_response(data):
                    return {"success": True, "data": data}
            return {"success": False, "error": "JSON 형식 문제"}
        except Exception as e:
            return {"success": False, "error": str(e)}

    def validate_response(self, data):
        """응답 검증"""
        required = ["monthly_purchase_probability", "monthly_purchase_frequency", "reasoning", "confidence_level"]
        if not all(field in data for field in required):
            return False
        if len(data["monthly_purchase_probability"]) != 12 or len(data["monthly_purchase_frequency"]) != 12:
            return False
        return True

    def save_intermediate_results(self, batch_name):
        """중간 결과 저장"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"/content/drive/MyDrive/Dongwon/result/medium_sim_{batch_name}_{timestamp}.json"

        save_data = {
            "summary": {
                "total_processed": len(self.results),
                "total_cost": self.total_cost,
                "failed_calls": self.failed_calls,
                "timestamp": timestamp
            },
            "results": self.results
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(save_data, f, ensure_ascii=False, indent=2)

        print(f"💾 중간 결과 저장: {filename}")

In [15]:
# 3. 시스템 초기화
prompt_generator = MediumScalePromptGenerator()
llm_client = MediumScaleLLMClient()

print("✅ LLM 시뮬레이션 시스템 준비 완료")

✅ LLM 시뮬레이션 시스템 준비 완료


## 실행 및 모니터링

In [16]:
# ===== 중규모 시뮬레이션 실행 =====
print("\n🚀 중규모 시뮬레이션 실행")
print("="*60)

# 1. 시뮬레이션 대상 준비
simulation_targets = []

# 각 카테고리에서 대표 제품 1개씩 선택
representative_products = {
    "조미소스": "동원참치액 순 500g",
    "참치": "동원맛참 고소참기름 135g",
    "우유류": "덴마크 하이그릭요거트 400g",
    "축산캔": "리챔 오믈레햄 200g"
}

for category, product_name in representative_products.items():
    personas = medium_personas[category]
    for persona in personas:
        simulation_targets.append({
            "persona": persona,
            "product_name": product_name,
            "category": category
        })

print(f"📊 시뮬레이션 대상: {len(simulation_targets)}개")
print(f"  - 제품별 페르소나: 25개")
print(f"  - 총 제품 수: 4개")
print(f"  - 예상 비용: ${len(simulation_targets) * 0.25:.2f}")
print(f"  - 예상 시간: {len(simulation_targets) * 3 / 60:.1f}분")

# 2. 프롬프트 생성
print(f"\n📝 프롬프트 생성 중...")
all_prompts = []

for target in simulation_targets:
    prompt = prompt_generator.create_prompt(
        target["persona"],
        target["product_name"],
        target["category"]
    )
    all_prompts.append(prompt)

print(f"✅ {len(all_prompts)}개 프롬프트 생성 완료")

# 3. 실행 확인
print(f"\n⚠️ 시뮬레이션 실행 전 확인:")
print(f"  • 총 LLM 호출: {len(all_prompts)}회")
print(f"  • 예상 비용: ${len(all_prompts) * 0.25:.2f}")
print(f"  • 예상 시간: {len(all_prompts) * 3 / 60:.1f}분")
print(f"  • API 키 설정: ✅")

proceed = input("계속 진행하시겠습니까? (y/n): ")

if proceed.lower() == 'y':
    print(f"\n🚀 시뮬레이션 시작!")
    start_time = datetime.now()

    # 배치 처리 실행
    results = llm_client.call_llm_batch(all_prompts, batch_size=10)

    end_time = datetime.now()
    duration = end_time - start_time

    print(f"\n🎉 시뮬레이션 완료!")
    print(f"="*60)
    print(f"실행 시간: {duration}")
    print(f"성공률: {len(results)}/{len(all_prompts)} ({len(results)/len(all_prompts)*100:.1f}%)")
    print(f"총 비용: ${llm_client.total_cost:.2f}")
    print(f"실패 횟수: {llm_client.failed_calls}")

else:
    print("❌ 시뮬레이션 중단")


🚀 중규모 시뮬레이션 실행
📊 시뮬레이션 대상: 100개
  - 제품별 페르소나: 25개
  - 총 제품 수: 4개
  - 예상 비용: $25.00
  - 예상 시간: 5.0분

📝 프롬프트 생성 중...
✅ 100개 프롬프트 생성 완료

⚠️ 시뮬레이션 실행 전 확인:
  • 총 LLM 호출: 100회
  • 예상 비용: $25.00
  • 예상 시간: 5.0분
  • API 키 설정: ✅
계속 진행하시겠습니까? (y/n): y

🚀 시뮬레이션 시작!
📞 총 100개 프롬프트를 10개씩 배치 처리

📦 배치 1: 10개 처리 중...
  [1/100] 조미소스 처리 중... ✅ 성공 ($0.0013)
  [2/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [3/100] 조미소스 처리 중... ✅ 성공 ($0.0015)
  [4/100] 조미소스 처리 중... ✅ 성공 ($0.0015)
  [5/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [6/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [7/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [8/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [9/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [10/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
⏸️ 배치 완료. 5초 휴식...
💾 중간 결과 저장: /content/drive/MyDrive/Dongwon/result/medium_sim_batch_1_20250822_170934.json

📦 배치 2: 10개 처리 중...
  [11/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [12/100] 조미소스 처리 중... ✅ 성공 ($0.0013)
  [13/100] 조미소스 처리 중... ✅ 성공 ($0.0015)
  [14/100] 조미소스 처리 중... ✅ 성공 ($0.0014)
  [15/100] 조미소스 처리 중... ✅ 성공 

## 중규모 시뮬레이션 결과 분석

In [17]:
# ===== 중규모 시뮬레이션 결과 분석 =====
print("\n📊 중규모 시뮬레이션 결과 분석")
print("="*80)

# 1. 기본 통계
total_results = len(llm_client.results)
success_rate = (total_results / len(all_prompts)) * 100

print(f"✅ 기본 통계:")
print(f"  • 총 호출: {len(all_prompts)}회")
print(f"  • 성공: {total_results}회")
print(f"  • 성공률: {success_rate:.1f}%")
print(f"  • 총 비용: ${llm_client.total_cost:.2f}")
print(f"  • 평균 응답시간: {llm_client.total_time/len(all_prompts):.1f}초")

# 2. 카테고리별 분석
category_analysis = {}

for result in llm_client.results:
    category = result["metadata"]["category"]
    response = result["response"]

    if category not in category_analysis:
        category_analysis[category] = {
            "count": 0,
            "avg_probability": [],
            "avg_frequency": [],
            "confidence_scores": [],
            "total_sales_estimates": []
        }

    # 통계 누적
    category_analysis[category]["count"] += 1

    monthly_prob = response["monthly_purchase_probability"]
    monthly_freq = response["monthly_purchase_frequency"]

    avg_prob = sum(monthly_prob) / 12
    avg_freq = sum(monthly_freq) / 12
    total_estimated_sales = sum([p/100 * f for p, f in zip(monthly_prob, monthly_freq)])

    category_analysis[category]["avg_probability"].append(avg_prob)
    category_analysis[category]["avg_frequency"].append(avg_freq)
    category_analysis[category]["confidence_scores"].append(response["confidence_level"])
    category_analysis[category]["total_sales_estimates"].append(total_estimated_sales)

print(f"\n📈 카테고리별 예측 패턴:")
for category, analysis in category_analysis.items():
    avg_prob = np.mean(analysis["avg_probability"])
    avg_freq = np.mean(analysis["avg_frequency"])
    avg_conf = np.mean(analysis["confidence_scores"])
    avg_sales = np.mean(analysis["total_sales_estimates"])

    print(f"\n[{category}] (n={analysis['count']})")
    print(f"  평균 구매확률: {avg_prob:.1f}%")
    print(f"  평균 구매빈도: {avg_freq:.2f}회/월")
    print(f"  평균 신뢰도: {avg_conf:.2f}")
    print(f"  예상 연간 구매: {avg_sales:.2f}회")

# 3. SMAPE 최적화 평가
print(f"\n🎯 SMAPE 최적화 평가:")

expected_conservatism = {
    "조미소스": {"level": "moderate", "expected_range": (15, 35)},
    "참치": {"level": "high", "expected_range": (10, 25)},
    "우유류": {"level": "very_high", "expected_range": (5, 20)},
    "축산캔": {"level": "extreme", "expected_range": (2, 15)}
}

for category, analysis in category_analysis.items():
    avg_prob = np.mean(analysis["avg_probability"])
    expected = expected_conservatism[category]
    min_exp, max_exp = expected["expected_range"]

    if min_exp <= avg_prob <= max_exp:
        assessment = "✅ 적절한 보수성"
    elif avg_prob > max_exp:
        assessment = "⚠️ 너무 낙관적"
    else:
        assessment = "⚠️ 너무 보수적"

    print(f"  [{category}] {avg_prob:.1f}% - {assessment}")

# 4. 풀 시뮬레이션 예산 산정
full_simulation_calls = 200 * 15  # 200 페르소나 × 15 제품
estimated_full_cost = full_simulation_calls * (llm_client.total_cost / len(all_prompts))
estimated_full_time = full_simulation_calls * (llm_client.total_time / len(all_prompts)) / 3600

print(f"\n💰 풀 시뮬레이션 예산 산정:")
print(f"  • 총 호출 예상: {full_simulation_calls:,}회")
print(f"  • 예상 비용: ${estimated_full_cost:.2f}")
print(f"  • 예상 시간: {estimated_full_time:.1f}시간")

# 5. 추천사항
print(f"\n🚀 다음 단계 추천:")

if success_rate == 100 and all(np.mean(analysis["confidence_scores"]) > 0.3 for analysis in category_analysis.values()):
    print("  ✅ 풀 시뮬레이션 진행 권장")
    print("  📊 안정적인 성능과 합리적인 예측 패턴 확인")
    print("  💡 추가 최적화 없이 바로 진행 가능")
else:
    print("  ⚠️ 프롬프트 개선 후 풀 시뮬레이션 권장")

print(f"\n🎯 최종 결론:")
print(f"  중규모 시뮬레이션 성공 → 풀 시뮬레이션 준비 완료!")


📊 중규모 시뮬레이션 결과 분석
✅ 기본 통계:
  • 총 호출: 100회
  • 성공: 100회
  • 성공률: 100.0%
  • 총 비용: $0.14
  • 평균 응답시간: 3.1초

📈 카테고리별 예측 패턴:

[조미소스] (n=25)
  평균 구매확률: 10.9%
  평균 구매빈도: 0.18회/월
  평균 신뢰도: 0.71
  예상 연간 구매: 0.41회

[참치] (n=25)
  평균 구매확률: 12.6%
  평균 구매빈도: 0.28회/월
  평균 신뢰도: 0.74
  예상 연간 구매: 0.66회

[우유류] (n=25)
  평균 구매확률: 10.4%
  평균 구매빈도: 0.10회/월
  평균 신뢰도: 0.73
  예상 연간 구매: 0.22회

[축산캔] (n=25)
  평균 구매확률: 10.5%
  평균 구매빈도: 0.15회/월
  평균 신뢰도: 0.77
  예상 연간 구매: 0.33회

🎯 SMAPE 최적화 평가:
  [조미소스] 10.9% - ⚠️ 너무 보수적
  [참치] 12.6% - ✅ 적절한 보수성
  [우유류] 10.4% - ✅ 적절한 보수성
  [축산캔] 10.5% - ✅ 적절한 보수성

💰 풀 시뮬레이션 예산 산정:
  • 총 호출 예상: 3,000회
  • 예상 비용: $4.19
  • 예상 시간: 2.6시간

🚀 다음 단계 추천:
  ✅ 풀 시뮬레이션 진행 권장
  📊 안정적인 성능과 합리적인 예측 패턴 확인
  💡 추가 최적화 없이 바로 진행 가능

🎯 최종 결론:
  중규모 시뮬레이션 성공 → 풀 시뮬레이션 준비 완료!
